In [ ]:
# http://www.donnees.statistiques.developpement-durable.gouv.fr/donneesCLC/CLC/region/CLC_RALPC_RGF_SHP.zip

In [ ]:
%load_ext lab_black

In [ ]:
import pandas as pd

clc_legend = pd.read_csv("../data/clc_legend.csv", dtype=str).assign(
    RGB=lambda df: "rgb(" + df["RGB"].str.split("-").str.join(", ") + ")"
)

In [ ]:
from geopandas import GeoDataFrame

corine12 = GeoDataFrame.from_file(
    "../data/CLC_RALPC_RGF_SHP/CLC12/CLC12_RALPC_RGF.shp"
)

In [ ]:
from cartes.osm import Nominatim, Overpass

landes = Nominatim.search("Landes")

In [ ]:
wgs = corine12.set_crs(2154).to_crs(4326)
intersection = wgs.intersects(landes.shape)
full_data = wgs[intersection].assign(geometry=lambda x: x.intersection(landes.shape))

In [ ]:
full_data = full_data.merge(clc_legend, left_on="CODE_12", right_on="CLC_CODE")

In [ ]:

towns = Overpass.request(area="Landes", node=dict(place="town"))
villages = Overpass.request(area="Landes", node=dict(place="village"))

cities = (
    pd.concat([towns.data, villages.data])
    .query("population.notnull()")
    .assign(population=lambda df: df.population.astype(int))
    .sort_values("population", ascending=False)[
        ["latitude", "longitude", "name", "population"]
    ]
)
cities

In [ ]:
import altair as alt


colors = sorted(full_data.CLC_CODE.unique())
domain = list(clc_legend.query("CLC_CODE in @colors").LABEL3)
range_ = list(clc_legend.query("CLC_CODE in @colors").RGB)

alt.data_transformers.disable_max_rows()

chart = (
    alt.layer(
        alt.Chart(full_data)
        .mark_geoshape()
        .encode(
            alt.Color(
                "LABEL3",
                legend=alt.Legend(
                    title="CORINE Land Cover", orient="bottom", columns=3
                ),
                scale=alt.Scale(domain=domain, range=range_),
            )
        ),
        alt.Chart(cities.head(7))
        .mark_text(font="Fira Sans", fontSize=14, align="left")
        .encode(
            alt.Latitude("latitude:Q"), alt.Longitude("longitude:Q"), alt.Text("name:N")
        ),
    )
    .project("conicConformal", rotate=[-3, -46.5], parallels=[49, 44], pointRadius=2)
    .properties(
        width=400,
        height=400,
        title="Landcover in the Landes departement",
    )
    .configure_title(font="Fira Sans", fontSize=18, anchor="start")
    .configure_legend(
        labelFont="Fira Sans", labelFontSize=13, titleFont="Fira Sans", titleFontSize=15
    )
    .configure_view(stroke=None)
)
chart

In [ ]:
chart.save("../contributions/challenge_day17.svg")